## AIHub Json Preprocessing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [ ]:
%pip install ray

In [1]:
import re
import os
import kss
import ray
import json
import time
from time import sleep
from tqdm import tqdm
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [19]:
pwd

'D:\\AIHUB'

### AIHUB 전문분야 말뭉치

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=110)

#### Write Json Text to TXT File

In [ ]:
from data_preprocessing import make_train_valid_json_txt_file_path_list
from data_preprocessing import count_number_of_txt_file_with_batch_list
from data_preprocessing import write_jsontext_to_txt_file_with_batch_list

In [15]:
json_path_list = ['AIHUB_전문분야 말뭉치/Training/'+ '/**/*.json', 
                  'AIHUB_전문분야 말뭉치/Validation/'+ '/**/*.json']
txt_path_list = ["exploration/professional_corpus_pro/AIHUB_professional_corpus_train_", 
                 "exploration/professional_corpus_pro/AIHUB_professional_corpus_valid_"]
corpus_name = "professional_corpus"

In [16]:
train_json_file_list, valid_json_file_list, train_txt_file_path_list, valid_txt_file_path_list = \
    make_train_valid_json_txt_file_path_list(json_path_list, txt_path_list, corpus_name)

The number of file: 39


In [22]:
batch_size = 1000
corpus_name = 'professional_corpus'
the_number_of_train_txt_file, the_number_of_train_txt_file_list = count_number_of_txt_file_with_batch_list(train_json_file_list, batch_size, corpus_name)

Batch Size: 1000
The number of txt file: 809


In [23]:
source_file_by_batch_train_df = pd.read_excel('source_file_by_batch/professional_corpus_train.xlsx', engine='openpyxl')  
source_file_by_batch_train_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_01.j...,5104,6,NaN
1,1,AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_02.j...,5524,6,NaN
2,2,AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_03.j...,4860,5,NaN
3,3,AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_04.j...,5497,6,NaN
4,4,AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_05.j...,5153,6,NaN
5,5,AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_06.j...,4829,5,NaN
6,6,AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_07.j...,4562,5,NaN
7,7,AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_08.j...,5193,6,NaN
8,8,AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_09.j...,5580,6,NaN
9,9,AIHUB_전문분야 말뭉치/Training\[원천]전문분야_train\특허_10.j...,5033,6,NaN


In [ ]:
batch_size = 1000
corpus_name = 'professional_corpus'
write_jsontext_to_txt_file_with_batch_list(train_json_file_list, train_txt_file_path_list, batch_size, the_number_of_train_txt_file_list, corpus_name)

In [28]:
batch_size = 1000
corpus_name = 'professional_corpus'
the_number_of_valid_txt_file, the_number_of_valid_txt_file_list = count_number_of_txt_file_with_batch_list(valid_json_file_list, batch_size, corpus_name)

Batch Size: 1000
The number of txt file: 113


In [29]:
source_file_by_batch_valid_df = pd.read_excel('source_file_by_batch/professional_corpus_valid.xlsx', engine='openpyxl')  
source_file_by_batch_valid_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_전문분야 말뭉치/Validation\[원천]전문분야_valid\특허_19...,5131,6,NaN
1,2,AIHUB_전문분야 말뭉치/Validation\[원천]전문분야_valid\논문_1_...,12544,13,NaN
2,3,AIHUB_전문분야 말뭉치/Validation\[원천]전문분야_valid\판례_ca...,69613,70,NaN
3,4,AIHUB_전문분야 말뭉치/Validation\[원천]전문분야_valid\법령_l_...,23592,24,NaN


In [ ]:
batch_size = 1000
corpus_name = 'professional_corpus'
write_jsontext_to_txt_file_with_batch_list(valid_json_file_list, valid_txt_file_path_list, batch_size, the_number_of_valid_txt_file_list, corpus_name)

#### Preprocess TXT File

In [ ]:
from sentence_segmentation import preprocessing_text
from data_preprocessing import make_pro_post_txt_file_path_list
from data_preprocessing import merge_and_deduplicate_corpus_txt
from reading_data import reading_txt

In [ ]:
pro_corpus_path = "AIHUB_corpus/exploration/professional_corpus_pro/AIHUB_professional_corpus_" + "*.txt"
pro_post_xlsx_path = "pro_post_corpus_path/professional_corpus.xlsx"
pro_total_corpus_path_list, post_total_corpus_path_list = make_pro_post_txt_file_path_list(pro_corpus_path, pro_post_xlsx_path)

In [38]:
len(pro_total_corpus_path_list)

922

In [40]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "source"

reading_txt(pro_coprus_file, line_length, data_type)

본 발명에 따른 표시 장치는 기판, 기판 위에 형성되어 있는 제1 보조 전극, 제1 보조 전극과 위에 형성되어 있고, 제1 입력 단자, 제1 출력 단자 및 제1 제어 단자를 가지는 제1 박막 트랜지스터, 제1 박막 트랜지스터와 연결되어 있는 화소 전극을 포함하고, 제1 보조 전극은 제1 박막 트랜지스터와 중첩하고, 제1 입력 단자와 전기적으로 연결되어 있다.



In [41]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "preprocessing"

reading_txt(pro_coprus_file, line_length, data_type)

본 발명에 따른 표시 장치는 기판, 기판 위에 형성되어 있는 제1 보조 전극, 제1 보조 전극과 위에 형성되어 있고, 제1 입력 단자, 제1 출력 단자 및 제1 제어 단자를 가지는 제1 박막 트랜지스터, 제1 박막 트랜지스터와 연결되어 있는 화소 전극을 포함하고, 제1 보조 전극은 제1 박막 트랜지스터와 중첩하고, 제1 입력 단자와 전기적으로 연결되어 있다.



In [52]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/professional_corpus.xlsx' \
    --start_index 0

2023-05-31 19:56:45,726	INFO worker.py:1625 -- Started a local Ray instance.


In [51]:
ray.shutdown()

In [ ]:
preprocessing_corpus_path = "AIHUB_corpus/exploration/professional_corpus_source_post/AIHUB_professional_corpus_source_" +"*.txt"
merge_corpus_path = 'AIHUB_corpus/duplicate/AIHUB_professional_corpus_source.txt'
deduplicate_corpus_path = 'AIHUB_corpus/AIHUB_professional_corpus_source.txt'

In [ ]:
merge_and_deduplicate_corpus_txt(preprocessing_corpus_path, merge_corpus_path, deduplicate_corpus_path)